In [1]:
import math
import collections
import pickle
import random
import scipy
import json

from matplotlib import pyplot as plt

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm


### 1. Perform conversions from volumes and carbon source name to design space

In [2]:
main_file_path = '/Users/harini/Dropbox (MIT)/Harini Narayanan’s files/1_Home/Biotech/MediaOptimization/BO_MediaOptimization/Molecules/HSA/HSA_TL_Additives_V2/'
carbon_source_filepath = '/Users/harini/Dropbox (MIT)/Harini Narayanan’s files/1_Home/Biotech/MediaOptimization/BO_MediaOptimization/Molecules/'

Stock_solid = pd.read_excel(carbon_source_filepath+'CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel(carbon_source_filepath +'CarbonSourceInfo.xlsx', 'Stocks_liquid')

CARBON NAME

In [3]:
Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])

UPPER BOUNDS

In [4]:
Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]

Glu_Ub = 50 #mM
Tween_Ub = 1# 1%
pH_Ub = 6.5# 5X

Glu_Stock_Conc = 250 #mM
Tween_Stock_Conc = 50# 1%
pH_Stock_Conc = 1# 

STOCK CONCENTRATIONS

In [5]:
Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()

In [6]:
Exp_Used = pd.read_csv('Actual_Round3_3mL.csv')
initN = Exp_Used.shape[0]
print(initN)
Exp_Used_CT = Exp_Used["Carbon_Type"].values
Exp_Used_CC = Exp_Used["Conc_Carbon [uL]"].values
Exp_Used_Gly = Exp_Used["Gly_OG [uL]"].values
Exp_Used_Met = Exp_Used["Met_Prod [uL]"].values

11


In [7]:
Exp_Used

,Unnamed: 0,Carbon_Type,Conc_Carbon [uL],Gly_OG [uL],Met_Prod [uL],Glu_OG [uL],Tween_OG [uL],Glu_Prod [uL],Tween_Prod [uL],pH
0,0,Xylose,2,51,289,41,18,341,12,6.00
1,1,Succinate,115,132,178,246,26,176,2,6.50
2,2,D-Arabinose,6,116,239,135,11,277,18,6.00
3,3,Trehalose,21,8,141,109,37,187,49,6.00
4,4,Galactose,177,61,170,14,13,349,14,5.75
5,5,Fructose,22,373,64,158,23,249,59,6.50
6,6,Sucrose,340,69,233,436,39,281,5,6.50
7,7,Ribose,158,85,143,234,50,274,44,6.00
8,8,Rhamnose,282,73,100,319,41,238,22,6.50
9,9,Galactose,487,157,247,49,10,356,16,6.00


In [8]:
Des_recon_CT = []
Des_recon_CC = []
Des_recon_Gly = []
Des_recon_Met = []

Glu_OG = []
Tween_OG = []
Glu_Prod = []
Tween_Prod = []
pH = []

for i in range(initN):
    index = Carbon_Names.index(Exp_Used_CT[i])
    Des_recon_CT.append(index)
    temp_factor = Carbon_Ub[index]/ Stock_Conc[index]
    cc = Exp_Used_CC[i]/ (temp_factor * 3)
    Des_recon_CC.append(cc)
    
    gly = Exp_Used_Gly[i] * OG_Stock_Conc/(OG_Gly_Ub * 3)
    Des_recon_Gly.append(gly)
    
    met = Exp_Used_Met[i]/( Met_Ub * 3)
    Des_recon_Met.append(met)
    
    Glu_OG.append(Exp_Used["Glu_OG [uL]"].values[i] * Glu_Stock_Conc/(Glu_Ub * 3 * 1000))
    Tween_OG.append(Exp_Used["Tween_OG [uL]"].values[i]* Tween_Stock_Conc/(Tween_Ub * 3 * 1000)) 
    Glu_Prod.append(Exp_Used["Glu_Prod [uL]"].values[i] * Glu_Stock_Conc/(Glu_Ub * 3 * 1000))
    Tween_Prod.append(Exp_Used["Tween_Prod [uL]"].values[i]* Tween_Stock_Conc/(Tween_Ub * 3 * 1000))
    pH.append(Exp_Used["pH"].values[i] * pH_Stock_Conc /(pH_Ub))

In [9]:
Design_actual = {'Carbon_Type': Des_recon_CT,
               'Conc_Carbon': Des_recon_CC,
               'Gly_OG': Des_recon_Gly,
               'Met_Prod': Des_recon_Met,
                'Glu_OG': Glu_OG,
                'Tween_OG': Tween_OG,
                'Glu_Prod' : Glu_Prod,
                'Tween_Prod': Tween_Prod,
                'pH': pH}

In [10]:
pd.DataFrame(Design_actual).to_csv('Reconstructed_Round3.csv')

In [ ]:
Carbon_Names